# Iris データ事前処理

In [1]:
# coding: UTF-8

import chainer
from chainer import Variable, Chain, optimizers
import chainer.links as L
import chainer.functions as F

import numpy as np
from sklearn import datasets # Scikit Learn にIris dataset が含まれている

In [2]:
# Iris データの読み込み
iris_data = datasets.load_iris()

In [3]:
x = iris_data.data.astype(np.float32)
t = iris_data.target
n = t.size

In [6]:
# 教師データの下処理
t_matrix = np.zeros(3 * n).reshape(n, 3).astype(np.float32)
for i in range(n):
    t_matrix[i, t[i]] = 1.0

print(t_matrix)

[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1. 

In [7]:
# 訓練用データとテスト用データ 半分が訓練用データで残りがテスト用データ
indexes = np.arange(n)
indexes_train = indexes[indexes%2 != 0]
indexes_test = indexes[indexes%2 == 0]

print(indexes)
print(indexes_train)
print(indexes_test)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149]
[  1   3   5   7   9  11  13  15  17  19  21  23  25  27  29  31  33  35
  37  39  41  43  45  47  49  51  53  55  57  59  61  63  65  67  69  71
  73  75  77  79  81  83  85  87  89  91  93  95  97  99 101 103 105 107
 109 111 113 115 117 119 121 123 125 127 129 131 133 135 137 139 141 143
 145 147 149]
[  0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34
  36  38  4

In [8]:
x_train = x[indexes_train, :] # 訓練用 入力
t_train = t_matrix[indexes_train, :] # 訓練用 正解
x_test = x[indexes_test, :] # テスト用 入力
t_test = t[indexes_test] # テスト用 正解

print(x_train)
print(t_train)
print(x_test)
print(t_test)

[[ 4.9000001   3.          1.39999998  0.2       ]
 [ 4.5999999   3.0999999   1.5         0.2       ]
 [ 5.4000001   3.9000001   1.70000005  0.40000001]
 [ 5.          3.4000001   1.5         0.2       ]
 [ 4.9000001   3.0999999   1.5         0.1       ]
 [ 4.80000019  3.4000001   1.60000002  0.2       ]
 [ 4.30000019  3.          1.10000002  0.1       ]
 [ 5.69999981  4.4000001   1.5         0.40000001]
 [ 5.0999999   3.5         1.39999998  0.30000001]
 [ 5.0999999   3.79999995  1.5         0.30000001]
 [ 5.0999999   3.70000005  1.5         0.40000001]
 [ 5.0999999   3.29999995  1.70000005  0.5       ]
 [ 5.          3.          1.60000002  0.2       ]
 [ 5.19999981  3.5         1.5         0.2       ]
 [ 4.69999981  3.20000005  1.60000002  0.2       ]
 [ 5.4000001   3.4000001   1.5         0.40000001]
 [ 5.5         4.19999981  1.39999998  0.2       ]
 [ 5.          3.20000005  1.20000005  0.2       ]
 [ 4.9000001   3.0999999   1.5         0.1       ]
 [ 5.0999999   3.4000001   1.5 

In [10]:
x_train_v = Variable(x_train)
t_train_v = Variable(t_train)
x_test_v = Variable(x_test)